In [5]:
# pip install openpyxl
import requests
import zipfile
import io
import pandas as pd

import json

## Extraktion Wahlomat Datensatz

In [10]:
# URL des ZIP-Archivs
zip_url = "https://www.bpb.de/system/files/datei/Wahl-O-Mat_Bundestagswahl_2025_Datensatz_v1.01.zip"

# HTTP-Request zum Herunterladen des ZIP-Archivs
response = requests.get(zip_url)

# Überprüfung, ob der Download erfolgreich war
if response.status_code == 200:
    # ZIP-Archiv im Speicher öffnen
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        # Den Namen der Excel-Datei abrufen
        excel_filename = [name for name in z.namelist() if name.endswith('.xlsx')][0]

        # Die gewünschte Arbeitsmappe "Datensatz BTW 2025" einlesen
        with z.open(excel_filename) as excel_file:
            df_wahlomat = pd.read_excel(excel_file, sheet_name="Datensatz BTW 2025")

    # DataFrame ausgeben
    display(df_wahlomat)

else:
    print(f"Fehler beim Abrufen der Datei: {response.status_code}")


,Partei: Nr.,Partei: Kurzbezeichnung,Partei: Name,These: Nr.,These: Titel,These: These,Position: Position,Position: Begründung
0,1,SPD,Sozialdemokratische Partei Deutschlands,1,Unterstützung der Ukraine,Deutschland soll die Ukraine weiterhin militär...,stimme zu,"Die SPD bekennt sich klar zur diplomatischen, ..."
1,2,CDU / CSU,Christlich Demokratische Union Deutschlands / ...,1,Unterstützung der Ukraine,Deutschland soll die Ukraine weiterhin militär...,stimme zu,Unser Ziel ist Frieden in Europa. Die Ukraine ...
2,3,GRÜNE,BÜNDNIS 90/DIE GRÜNEN,1,Unterstützung der Ukraine,Deutschland soll die Ukraine weiterhin militär...,stimme zu,Millionen Ukrainer*innen verteidigen Tag für T...
3,4,FDP,Freie Demokratische Partei,1,Unterstützung der Ukraine,Deutschland soll die Ukraine weiterhin militär...,stimme zu,Unsere Unterstützung für die Ukraine darf nich...
4,5,AfD,Alternative für Deutschland,1,Unterstützung der Ukraine,Deutschland soll die Ukraine weiterhin militär...,stimme nicht zu,Wir fordern von allen Kriegsparteien eine Eins...
...,...,...,...,...,...,...,...,...
1059,24,BüSo,Bürgerrechtsbewegung Solidarität,38,Erhöhung des Mindestlohns,Der gesetzliche Mindestlohn soll spätestens 20...,stimme zu,Jeder sollte von seiner Arbeit leben können. F...
1060,25,BÜNDNIS DEUTSCHLAND,BÜNDNIS DEUTSCHLAND,38,Erhöhung des Mindestlohns,Der gesetzliche Mindestlohn soll spätestens 20...,stimme nicht zu,Die Mindestlohnregelungen stellen einen gravie...
1061,26,BSW,Bündnis Sahra Wagenknecht – Vernunft und Gerec...,38,Erhöhung des Mindestlohns,Der gesetzliche Mindestlohn soll spätestens 20...,stimme zu,NaN
1062,27,MERA25,MERA25 - Gemeinsam für Europäische Unabhängigkeit,38,Erhöhung des Mindestlohns,Der gesetzliche Mindestlohn soll spätestens 20...,stimme zu,Wir setzen uns ein für einen bundesweiten effe...


## Extraktion Real-O-Mat Datensatz

In [12]:
# Manueller Download von Website  https://real-o-mat.de/methodik/


# Datei einlesen
with open("real-o-mat.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Extrahieren der relevanten Daten aus dem JSON
questions = []
for entry in data["data"]:
    question_id = entry["id"]
    thesis = entry["thesis"]
    category = entry["category"]
    
    for answer in entry["answers"]:
        party = answer["party"]
        answer_text = answer["answer"]
        comment = answer.get("comment", "")  # Falls kein Kommentar vorhanden ist, leer setzen
        
        questions.append({
            "Frage-ID": question_id,
            "These": thesis,
            "Kategorie": category,
            "Partei": party,
            "Antwort": answer_text,
            "Kommentar": comment
        })

# DataFrame erstellen
df_realomat = pd.DataFrame(questions)

# DataFrame anzeigen
display(df_realomat)


,Frage-ID,These,Kategorie,Partei,Antwort,Kommentar
0,question-0,"Bürgergeldempfänger*innen, die wiederholt eine...",Arbeit,spd,richtig,"Die SPD-Fraktion stimmt dem <a href=""https://d..."
1,question-0,"Bürgergeldempfänger*innen, die wiederholt eine...",Arbeit,cdu,nicht weit genug,"Die Unionsfraktion lehnt den <a href=""https://..."
2,question-0,"Bürgergeldempfänger*innen, die wiederholt eine...",Arbeit,fdp,richtig,"Die FPD-Fraktion stimmt dem <a href=""https://d..."
3,question-0,"Bürgergeldempfänger*innen, die wiederholt eine...",Arbeit,gruene,richtig,Die Fraktion BÜNDNIS 90/DIE GRÜNEN stimmt dem ...
4,question-0,"Bürgergeldempfänger*innen, die wiederholt eine...",Arbeit,bsw,/,"Die Gruppe BSW lehnt den <a href=""https://dser..."
...,...,...,...,...,...,...
135,question-19,Mit der Subventionierung von Agrardiesel solle...,Landwirtschaft,fdp,zu weit,"Die FDP-Fraktion lehnt den <a href=""https://ds..."
136,question-19,Mit der Subventionierung von Agrardiesel solle...,Landwirtschaft,gruene,zu weit,Die Fraktion BÜNDNIS 90/DIE GRÜNEN lehnt den <...
137,question-19,Mit der Subventionierung von Agrardiesel solle...,Landwirtschaft,bsw,/,"Die Gruppe BSW lehnt den <a href=""https://dser..."
138,question-19,Mit der Subventionierung von Agrardiesel solle...,Landwirtschaft,linke,zu weit,"Die Gruppe Die Linke lehnt den <a href=""https:..."


In [20]:
df_realomat.Kategorie.unique()

13

In [18]:
df_wahlomat["These: Titel"].unique()

array(['Unterstützung der Ukraine', 'Erneuerbare Energien',
       'Streichung des Bürgergelds', 'Tempolimit auf Autobahnen',
       'Abweisung Asylsuchender', 'Begrenzung der Mietpreise',
       'Automatisierte Gesichtserkennung', 'Energieintensive Unternehmen',
       'Rente nach 40 Beitragsjahren', 'Grundgesetz',
       'Anwerbung von Fachkräften', 'Nutzung der Kernenergie',
       'Anhebung des Spitzensteuersatzes',
       'Kompetenzen in der Schulpolitik', 'Rüstungsexporte nach Israel',
       'Krankenkassen', 'Abschaffung der Frauenquote',
       'Ökologische Landwirtschaft', 'Projekte gegen Rechtsextremismus',
       'Kontrolle von Zulieferern', 'Elternabhängiges BAföG',
       'Schuldenbremse', 'Arbeitserlaubnis für Asylsuchende',
       'Verwerfen der Klimaziele', '35-Stunden-Woche',
       'Schwangerschaftsabbruch nach Beratung', 'Nationale Währung',
       'Schiene vor Straße', 'Ehrenamt', 'Umlegung der Grundsteuer',
       'Einschränkung des Streikrechts', 'Volksentscheide'

In [23]:
print(f"Anzahl Kategorien Real-o-Mat: ", len(df_realomat.Kategorie.unique()))
print(f"Anzahl Thesentitel Wahl-o-Mat: ", len(df_wahlomat["These: Titel"].unique()))

Anzahl Kategorien Real-o-Mat:  13
Anzahl Thesentitel Wahl-o-Mat:  38
